<a href="https://colab.research.google.com/github/AndySAnker/ClusterFinder/blob/main/ClusterFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ClusterFinder
**Github:** https://github.com/AndySAnker/ClusterFinder

**Paper:** ClusterFinder: XXXX

**Questions:** andy@chem.ku.dk 

Welcome to ClusterFinder that is a XXXX

This script guides you through a simple example of how to use ClusterFinder to find a cluster from a given PDF.

# First install new libstdc version

In [1]:
%%capture
!sudo add-apt-repository -y ppa:ubuntu-toolchain-r/test
!sudo apt-get update
!sudo apt-get -y install gcc-4.9
!sudo apt-get upgrade libstdc++6

get_ipython().kernel.do_shutdown(True)

# Download anaconda

In [1]:
%%capture
%%bash 
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX


# Download DiffPy-CMI

In [2]:
%%capture
!conda create -n diffpy -c defaults -c diffpy python=3.7 diffpy-cmi pandas --yes
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.srfit-3.0.0-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.structure-3.0.1-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/
!cp -r /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy.utils-3.0.0-py3.7.egg/diffpy/* /usr/local/envs/diffpy/lib/python3.7/site-packages/diffpy/

import sys
sys.path.insert(1, "/usr/local/envs/diffpy/lib/python3.7/site-packages")

# Import modules and functions

In [3]:
%%capture
!sudo apt-get -y install software-properties-common
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test
!sudo apt-get update
!sudo apt-get -y install gcc-4.9
!sudo apt-get upgrade libstdc++6

In [4]:
import os
if not os.path.exists("ClusterFinder"):
  os.mkdir("ClusterFinder")
if not os.path.exists("ClusterFinder/utils"):
  os.mkdir("ClusterFinder/utils")
!wget https://sid.erda.dk/share_redirect/bDf02Rk9jg
%cp bDf02Rk9jg "ClusterFinder/utils/Bonding.txt"

--2022-10-28 07:41:00--  https://sid.erda.dk/share_redirect/bDf02Rk9jg
Resolving sid.erda.dk (sid.erda.dk)... 130.225.104.13
Connecting to sid.erda.dk (sid.erda.dk)|130.225.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3216 (3.1K)
Saving to: ‘bDf02Rk9jg’

bDf02Rk9jg          100%[===================>]   3.14K  --.-KB/s    in 0s      

2022-10-28 07:41:00 (126 MB/s) - ‘bDf02Rk9jg’ saved [3216/3216]



In [5]:
%%capture
#!git clone https://github.com/AndySAnker/fast-MotEx.git
!apt install ase
import numpy as np
import matplotlib as mpl
import matplotlib.cm
import time, random
from scipy.optimize.minpack import leastsq
from diffpy.Structure import Structure, Atom
from diffpy.srfit.pdf import PDFContribution, PDFParser, PDFGenerator
from diffpy.srfit.fitbase import FitRecipe, FitResults, Profile, FitContribution
from diffpy.srreal.pdfcalculator import DebyePDFCalculator
from multiprocessing import Pool
from functools import partial
from ase.io import read
from google.colab import output, files

random.seed(14)
np.random.seed(14)

In [30]:
def structure_catalogue_maker(Number_of_atoms):
    """Makes a catalogue of structures"""
    
    structure_catalogue = np.ones((Number_of_atoms,Number_of_atoms))
    structure_catalogue[np.array([range(Number_of_atoms)]),np.array([range(Number_of_atoms)])] = 0
    return structure_catalogue

def Load(Experimental_Data, starting_model):
    """This function loads the data and structure"""
    
    # Get experimental data
    for skip_row in range(100):
        try:
            Exp_data = np.loadtxt(Experimental_Data, skiprows=skip_row)
        except ValueError:
            continue
    # Nyquist sampling the data by only using every 10nth datapoint
    if Exp_data[1,0] - Exp_data[0,0] < 0.1:
        Exp_r, Exp_Gr = Exp_data[::10,0], Exp_data[::10,1]
    else:
        Exp_r, Exp_Gr = Exp_data[:,0], Exp_data[:,1]
    # Normalise data
    Exp_Gr /= max(Exp_Gr)

    # Read structure and divide it into two lists: Atoms we want to iterate (W) and atoms we do not iterate (O)
    struct=[]
    with open(starting_model, 'r') as fi:
        for line in fi.readlines():
            sep_line=line.strip('{}\n\r ').split()
            if len(sep_line)==4: #  tillader andre informationer i xyz filen some ikke skal laeses
                struct.append(sep_line)
    elements=np.array(struct)[:,0]
    xyz=(np.array(struct)[:,1:].astype(float))
    
    return Exp_r, Exp_Gr, elements, xyz

def format_XYZ(starting_model, allowed_atoms):
	# Read structure and divide it into two lists: Atoms we want to iterate and atoms we do not iterate.
	# Save the file in this new format and get number of atoms that we iterate.
	permutable_struct = []
	nonpermutable_struct = []
	with open(starting_model, 'r') as fi:
		for line in fi.readlines():
			sep_line=line.strip('{}\n\r ').split()
			if len(sep_line)==4: #  tillader andre informationer i xyz filen some ikke skal laeses
				if sep_line[0] in allowed_atoms:
					permutable_struct.append(sep_line)
				else:
					nonpermutable_struct.append(sep_line)
	struct = permutable_struct + nonpermutable_struct

	New_XYZ = open(starting_model, "w")
	New_XYZ.write(str(len(struct)) + "\n\n")
	for i in range(len(struct)):
		New_XYZ.write(str(struct[i]).replace("[", "").replace("]", "").replace("'", "").replace(",", "") + "\n")
	New_XYZ.close()

	Num_permutable_atoms = len(permutable_struct)
	return Num_permutable_atoms

def fitting(structure_catalogue, xyz, atom_ph, Qmin, Qmax, Qdamp, rmin, rmax, plot, index):
    """This function takes in a 'starting_model', and an 'index' from the 'structure_catalogue'. It generates the 
    corresponding structure and fit it to the 'Experimental_Data'."""
    
    xyz_Mo = xyz[:NumMo].copy()
    xyz_O = xyz[NumMo:len(xyz)].copy()
    keep_O = np.zeros(len(xyz_O))
    # Cycle through W atoms and delete W according to index 0's from permutation
    delete_M = np.where(np.array(structure_catalogue)[index,:] == 0)[0]

    # Delete atoms from starting model 
    xyz_Mo = np.delete(xyz_Mo, delete_M, 0)

    # Cycle through all atoms that is not iteratable and test if it is within the threshold distance. Delete atoms with no bonds
    for j in range(len(xyz_O)):
        dists = np.sqrt((xyz_Mo[:,0]-xyz_O[j,0])**2+(xyz_Mo[:,1]-xyz_O[j,1])**2+(xyz_Mo[:,2]-xyz_O[j,2])**2)
        if np.min(dists) < threshold:    
            keep_O[j] = 1

    # Cycle through W atoms and delete W according to index 0's from permutation
    delete_O = np.where(np.array(keep_O) == 0)[0]
    # Delete atoms from starting model 
    xyz_O = np.delete(xyz_O, delete_O, 0)

    # Create structure for iterable (W) and non-iterable (O) atoms and combine them
    Mo_cluster = Structure([Atom(atom_ph, xi) for xi in xyz_Mo])
    O_cluster = Structure([Atom('O', xi) for xi in xyz_O])
    cluster = Mo_cluster + O_cluster
    
    # Make a standard cluster refinement using Diffpy-CMI
    # Import the data and make it a PDFprofile. Define the range of the data that will be used in the fit.
    pdfprofile = Profile()
    pdfparser = PDFParser()
    pdfparser.parseFile(Experimental_Data)
    pdfprofile.loadParsedData(pdfparser)
    pdfprofile.setCalculationRange(xmin = rmin, xmax = rmax)

    # Setup the PDFgenerator that calculates the PDF from the structure
    pdfgenerator_cluster = PDFGenerator("G")
    # Add the profile and both generators to the PDFcontribution
    pdfcontribution = FitContribution("pdf")
    pdfcontribution.setProfile(pdfprofile, xname="r") 
    pdfcontribution.addProfileGenerator(pdfgenerator_cluster)
    
    pdfgenerator_cluster.setQmin(Qmin)
    pdfgenerator_cluster.setQmax(Qmax)
    pdfgenerator_cluster._calc.evaluatortype = 'OPTIMIZED'
    pdfgenerator_cluster.setStructure(cluster, periodic = False)

    # Use scaling factors proportional to molar content
    pdfcontribution.setEquation('mc*G')

    # Define the recipe to do the fit and add it to the PDFcontribution
    recipe = FitRecipe()
    recipe.addContribution(pdfcontribution)

    # Avoid too much output during fitting 
    recipe.clearFitHooks()

    # Add the scale factor.
    recipe.addVar(pdfcontribution.mc, 1.0, tag = "scale")
    
    # Add the instrumental parameters to the two generators
    pdfgenerator_cluster.qdamp.value = Qdamp
    #pdfgenerator_cluster.qbroad.value = 0.00

    # Add the delta2 parameters, and make sure it cannot take unphysical values
    #recipe.addVar(pdfgenerator_cluster.delta2, 0, name = "delta2_cluster", tag = "delta2")
    #recipe.restrain("delta2_cluster", lb=0, ub = 7, sig=0.001);
    
    # Add ADP and "cell" for the cluster
    phase_cluster = pdfgenerator_cluster.phase
    atoms = phase_cluster.getScatterers()
    lat = phase_cluster.getLattice()

    recipe.newVar("zoomscale", 1, tag = "lat")
    recipe.constrain(lat.a, 'zoomscale')
    recipe.constrain(lat.b, 'zoomscale')
    recipe.constrain(lat.c, 'zoomscale')
    recipe.restrain("zoomscale", lb=0.95, ub = 1.05, sig=0.001)

    Mo_cluster = recipe.newVar("Mo_Biso_cluster", 0.3, tag = 'adp_Mo')
    O_cluster = recipe.newVar("O_Biso_cluster", 0.4, tag = 'adp_O')

    for atom in atoms:
        if atom.element.title() == atom_ph:
            recipe.constrain(atom.Biso, Mo_cluster)
        elif atom.element.title() == "O":
            recipe.constrain(atom.Biso, O_cluster)
    recipe.restrain("Mo_Biso_cluster", lb=0.08, ub = 3, sig=0.001)
    recipe.restrain("O_Biso_cluster", lb=0.08, ub = 3, sig=0.001)

    pos_limit1 = 0.05
    pos_limit2 = 0.02
    pos_limit3 = 0.02
    for atom in atoms:
        if atom.element == atom_ph:
            recipe.addVar(atom.x, name=atom.name+'_x1', tag='xyz1')
            recipe.addVar(atom.y, name=atom.name+'_y1', tag='xyz1')
            recipe.addVar(atom.z, name=atom.name+'_z1', tag='xyz1')       
            #restrain atomic positions
            lbx = atom.x.value-pos_limit1
            ubx = atom.x.value+pos_limit1
            recipe.restrain(atom.x, lb=lbx, ub = ubx, sig=0.001)
            lby = atom.y.value-pos_limit1
            uby = atom.y.value+pos_limit1
            recipe.restrain(atom.y, lb=lby, ub = uby, sig=0.001)
            lbz = atom.z.value-pos_limit1
            ubz = atom.z.value+pos_limit1
            recipe.restrain(atom.y, lb=lby, ub = uby, sig=0.001)
    
    #free parameters are set
    recipe.fix('all')
    recipe.free("scale")
    #leastsq(recipe.residual, recipe.getValues())
    recipe.free("lat")
    leastsq(recipe.residual, recipe.getValues())
    #recipe.free("adp_Mo")
    #leastsq(recipe.residual, recipe.getValues())
    #recipe.free("adp_O")
    #leastsq(recipe.residual, recipe.getValues())
    #recipe.free("xyz1")
    #leastsq(recipe.residual, recipe.getValues())
    
    # Turn off printout of iteration number.
    #recipe.clearFitHooks()
    
    # We calculate the goodness-of-fit, Rwp
    g = recipe.pdf.profile.y
    gcalc = recipe.pdf.evaluate()
    rfactor1 = np.sqrt(sum((g - gcalc)**2) / sum((g)**2))
    
    # if plot == 1 it will also plot the fit
    if plot == 1:
        print ("FIT RESULTS\n")
        res1 = FitResults(recipe)
        print (res1)

        # Plot the observed and refined PDF.
        # Get the experimental data from the recipe
        r = recipe.pdf.profile.x
        gobs = recipe.pdf.profile.y

        # Get the calculated PDF and compute the difference between the calculated and measured PDF
        gcalc = recipe.pdf.evaluate()
        baseline = 1.1 * gobs.min()
        gdiff = gobs - gcalc

        # Plot!
        plt.figure()
        plt.plot(r, gobs, 'bo', label="G(r) data")
        plt.plot(r, gcalc, 'r-', label="G(r) fit")
        plt.plot(r, gdiff + baseline, 'g-', label="G(r) diff")
        plt.plot(r, np.zeros_like(r) + baseline, 'k:')
        plt.xlabel(r"$r (\AA)$")
        plt.ylabel(r"$G (\AA^{-2})$")
        plt.legend()

        plt.show()
    return 1-rfactor1

def fitting_multiprocess(structure_catalogue, xyz, atom_ph, Qmin, Qmax, Qdamp, rmin, rmax, SaveName, cores=1):
    """This function runs the refinement of all the structures in the structure catalogue using multiprocessing"""
    start_time = time.time()
    values = []
    # Set up multiprocessing refinement
    fitindex = range(len(structure_catalogue))
    p = Pool(processes=cores)
    plot = 0
    
    func = partial(fitting, structure_catalogue, xyz, atom_ph, Qmin, Qmax, Qdamp, rmin, rmax, plot)
    results = p.map(func, fitindex)
    p.close()
    p.join()
    
    # Start refinement and append results to lists
    for i in fitindex:
        if i % 100 == 0:
            print ("I have now fitted: ", str(i) + " structures out of " + str(len(structure_catalogue)))
        rw = results[i]
        values.append(rw)
    values = np.reshape(values,(int(len(values)/1) , 1))
    
    # Save results in format that is suitable for Machine Learning
    Result = np.column_stack([values, np.asarray(structure_catalogue)])
    np.savetxt(SaveName, Result)
    return Result

def calculate_atomContributionValue(Result, saveResults):
    """Calculate atom contribution value list from the result array"""
    
    # Define AtomContributionValues vector
    AtomContributionValues = Result[:,0]
    
    # Normalise the AtomContributionValues
    amin, amax = min(AtomContributionValues), max(AtomContributionValues)
    AtomContributionValues = (AtomContributionValues - amin) / (amax - amin)
    AtomContributionValues_ph = AtomContributionValues.copy()
    AtomContributionValues_ph.sort()

    # Define colormap of viridis.reverse
    norm = mpl.colors.Normalize(vmin=AtomContributionValues_ph[round((len(AtomContributionValues))/10)], vmax=AtomContributionValues_ph[-round((len(AtomContributionValues))/10)])
    cmap = matplotlib.cm.cividis_r
    m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    
    # Save results to file
    f = open(saveResults+"AtomContributionValues_MotEx.txt", "w")
    f.write("\nAtom contribution are calculated to: \n")
    for i in range(len(AtomContributionValues)):
        f.write("Atom # "+ str(i+1) + ":  "+ str(AtomContributionValues[i]) + "  Colorcode:  "+ mpl.colors.rgb2hex(m.to_rgba(AtomContributionValues[i]))+"\n")
    
    return m, AtomContributionValues

def Make_CrystalMakerFile(elements, xyz, AtomContributionValues, m, saveResults, threshold):
    # Read bonds and colors of all atoms
    bonding = []
    with open("ClusterFinder/utils/Bonding.txt", 'r') as fi:
        for line in fi.readlines():
            sep_line=line.strip('{}\n\r ').split()
            bonding.append(sep_line)
    bonding = np.array(bonding)
    
    # Output a crystalmaker file to visualize the results
    CrystalMaker = open(saveResults + '_CrystalMaker.cmtx', 'w')

    CrystalMaker.write("MOLE  CrystalMaker molecule format\n")
    CrystalMaker.write("TITL  Molecule\n\n")
    CrystalMaker.write("! Model type\n")
    CrystalMaker.write("MODL  1\n\n")

    CrystalMaker.write("! Depth fading settings\n")
    CrystalMaker.write("DCUE  1.000000 0.212899 0.704686\n\n")

    CrystalMaker.write("! Colour definitions:\n")
    CrystalMaker.write("TYPE\n")

    # Assign colors to all the atoms
    for iter, element in enumerate(elements):
        if iter < NumMo:
            #CrystalMaker.write(element + str(iter+1) + " 1.32 ")
            CrystalMaker.write(element + str(iter+1) + " " + bonding[np.where(bonding == element)[0][0], 1] + " ")
            rgb1 = m.to_rgba(AtomContributionValues[iter])[:-1][0]
            rgb2 = m.to_rgba(AtomContributionValues[iter])[:-1][1]
            rgb3 = m.to_rgba(AtomContributionValues[iter])[:-1][2]
            CrystalMaker.write(str(rgb1) + " " + str(rgb2) + " " + str(rgb3))
            CrystalMaker.write("\n")
        else:
            #CrystalMaker.write(element + str(iter+1) + " 0.66 ")
            CrystalMaker.write(element + str(iter+1) + " " + bonding[np.where(bonding == element)[0][0], 1] + " ")
            rgb1 = int(float(bonding[np.where(bonding == element)[0][0], 2])*255) #mpl.colors.to_rgb("#FF0000")[0]
            rgb2 = int(float(bonding[np.where(bonding == element)[0][0], 3])*255) #mpl.colors.to_rgb("#FF0000")[1]
            rgb3 = int(float(bonding[np.where(bonding == element)[0][0], 4])*255) #mpl.colors.to_rgb("#FF0000")[2]
            CrystalMaker.write(str(rgb1) + " " + str(rgb2) + " " + str(rgb3))
            CrystalMaker.write("\n")
    
    CrystalMaker.write("\n")
    CrystalMaker.write("! Atoms list\n")
    CrystalMaker.write("! Bond Specifications\n")
    
    # Assign bonds between the atoms
    for iter, element in enumerate(elements):
        if iter < NumMo:
            NI_elements = np.delete(np.unique(elements), np.where(np.unique(elements) == element)[0])
            for NI_element in NI_elements:
                CrystalMaker.write("BMAX " + element + " " + str(NI_element) + "  " + str(threshold))
                CrystalMaker.write("\n")
    
    CrystalMaker.write("\n")
    CrystalMaker.write("! Atoms list\n")
    CrystalMaker.write("ATOM\n")
    
    # Assign coordinates to the atoms
    for iter, element in enumerate(elements):
        if iter < NumMo:
            CrystalMaker.write(element + " " + element + str(iter+1) + " " + str(xyz[iter][0]) + " " + str(xyz[iter][1]) + " " + str(xyz[iter][2]) + "\n")
        else:
            CrystalMaker.write(element + " " + element + str(iter+1) + " " + str(xyz[iter][0]) + " " + str(xyz[iter][1]) + " " + str(xyz[iter][2]) + "\n")

    CrystalMaker.close()
    
    return None

def Make_VestaFile(elements, xyz, AtomContributionValues, m, saveResults, threshold):
    # Read bonds and colors of all atoms
    bonding = []
    with open("ClusterFinder/utils/Bonding.txt", 'r') as fi:
        for line in fi.readlines():
            sep_line=line.strip('{}\n\r ').split()
            bonding.append(sep_line)
    bonding = np.array(bonding)

    # Output a Vesta file to visualize the results
    Vesta = open(saveResults + '_Vesta.vesta', 'w')

    Vesta.write("#VESTA_FORMAT_VERSION 3.5.4\n\n\n")
    Vesta.write("MOLECULE\n\n")
    Vesta.write("Title\n")
    Vesta.write("XYZ file\n\n")

    Vesta.write("STRUC\n")
    # Assign coordinates to the atoms
    for iter, element in enumerate(elements):
        Vesta.write(str(iter+1) + " " + element + " " + element + str(iter+1) + " 1.0000 " + str(xyz[iter][0]) + " " + str(xyz[iter][1]) + " " + str(xyz[iter][2]) + "1" + " -" + "\n")
        Vesta.write("0 0 0 0\n")
    Vesta.write("  0 0 0 0 0 0 0\n")

    Vesta.write("SBOND\n")
    # Assign bonds between the atoms
    unique_elements = np.unique(elements)
    for iter, element1 in enumerate(unique_elements):
      for iter, element2 in enumerate(unique_elements):
        if not element1 == element2:
          Vesta.write(str(iter+1) + " " + element1 + " " + element2 + " 0.0 " + str(threshold) + " 0 1 1 0 1 0.25 2 127 127 127\n")
          Vesta.write("0 0 0 0\n")
    
    Vesta.write("SITET\n")
    # Assign colors to all the atoms
    for iter, element in enumerate(elements):
        if iter < NumMo:
            rgb1 = int(m.to_rgba(AtomContributionValues[iter])[:-1][0]*255)
            rgb2 = int(m.to_rgba(AtomContributionValues[iter])[:-1][1]*255)
            rgb3 = int(m.to_rgba(AtomContributionValues[iter])[:-1][2]*255)
            Vesta.write(str(iter+1) + " " + element + str(iter+1) + " " + bonding[np.where(bonding == element)[0][0], 1] + " " + str(rgb1) + " " + str(rgb2) + " " + str(rgb3) + " " + str(rgb1) + " " + str(rgb2) + " " + str(rgb3) + " 204 0\n")
        else:
            rgb1 = int(float(bonding[np.where(bonding == element)[0][0], 2])*255)
            rgb2 = int(float(bonding[np.where(bonding == element)[0][0], 3])*255)
            rgb3 = int(float(bonding[np.where(bonding == element)[0][0], 4])*255)
            Vesta.write(str(iter+1) + " " + element + str(iter+1) + " " + bonding[np.where(bonding == element)[0][0], 1] + " " + str(rgb1) + " " + str(rgb2) + " " + str(rgb3) + " " + str(rgb1) + " " + str(rgb2) + " " + str(rgb3) + " 204 0\n")
    Vesta.write("0 0 0 0 0 0\n")
    
    Vesta.write("ATOMT\n")
    done_deal_atoms = []
    for iter, element in enumerate(elements):
      if element not in done_deal_atoms:
        rgb1 = int(float(bonding[np.where(bonding == element)[0][0], 2])*255)
        rgb2 = int(float(bonding[np.where(bonding == element)[0][0], 3])*255)
        rgb3 = int(float(bonding[np.where(bonding == element)[0][0], 4])*255)
        Vesta.write(str(iter+1) + " " + element + " " + bonding[np.where(bonding == element)[0][0], 1] + " " + str(rgb1) + " " + str(rgb2) + " " + str(rgb3) + " " + str(rgb1) + " " + str(rgb2) + " " + str(rgb3) + " 204\n")
        done_deal_atoms.append(element)
    Vesta.write("0 0 0 0 0 0\n")

    Vesta.close()
    
    return None


# Define variables

In [25]:
starting_model = list(files.upload())[0] # Name of the starting model file
StemName = list(files.upload())[0][:-3] # Upload PDF(s) from local computer
threshold = 2.6 # Longest bond distance between the metal and oxygen atom
allowed_atoms = ["Bi"] # The atoms that should be permuted, can be multiple atoms
#atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "W", 0.7, 20, 0.05, 1.6, 10
#atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "Al", 0.7, 25, 0.04, 1, 10
#atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "C", 0.5, 24, 0.02, 1, 15
atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "Bi", 0.7, 15.5, 0.04, 2, 15
#atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "Ce", 0.7, 24, 0.04, 1, 20


Saving icsd_010480_cleaned.xyz to icsd_010480_cleaned (3).xyz


Saving epsilon-keggin-170336-ICSD cutout.gr to epsilon-keggin-170336-ICSD cutout (4).gr


# Run

In [26]:
saveResults = "./"
start_time = time.time()

# Step 1: Make the structure catalogue
NumMo = format_XYZ(starting_model, allowed_atoms) # Format the starting model and calculate then number of atoms that should be permuted in the starting model
structure_catalogue = structure_catalogue_maker(Number_of_atoms=NumMo)
print ("Catalogue maker: ", time.time() - start_time)

# Step 2: Fit all of the structures from the catalogue of structure motifs to the dataset
### First define the experimental data path and the path you want the structure catalogue with fits to be saved
Experimental_Data = StemName+".gr" # Name of the experimental file
saveFits = StemName+".txt" # Name of the saved fits file

# Load data and start model
Exp_r, Exp_Gr, elements, xyz = Load(Experimental_Data, starting_model)
print ("Data loader: ", time.time() - start_time)

### Produce organized structure catalogue with Rwp values
Result = fitting_multiprocess(structure_catalogue, xyz, atom_ph, Qmin, Qmax, Qdamp, rmin, rmax, SaveName=saveFits, cores=None)
print ("Getting results: ", time.time() - start_time)

# Step 4: Calculate Atom Contribution values
m, AtomContributionValues = calculate_atomContributionValue(Result, saveResults)
print ("Calculating atom contribution values: ", time.time() - start_time)

Make_CrystalMakerFile(elements, xyz, AtomContributionValues, m, StemName, threshold)
Make_VestaFile(elements, xyz, AtomContributionValues, m, StemName, threshold)
print ("Output a CrystalMaker file: ", time.time() - start_time)

print (time.time() - start_time, " s")


Catalogue maker:  0.004961729049682617
Data loader:  0.20579051971435547
I have now fitted:  0 structures out of 24
Getting results:  8.077596187591553
Calculating atom contribution values:  8.08876371383667
Output a CrystalMaker file:  8.125550985336304
8.12581753730774  s


## Download results

In [27]:
# Download the latest results
files.download(StemName + '_CrystalMaker.cmtx')
files.download(StemName + '_Vesta.vesta')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
AtomContributionValues_ph = AtomContributionValues.copy()
AtomContributionValues_ph = sorted(AtomContributionValues)

AtomContributionValues_ph[24], np.where(AtomContributionValues < AtomContributionValues_ph[24]), AtomContributionValues

(0.38850327501217574,
 (array([ 1,  3,  5,  7, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37,
         39, 41, 43, 44, 45, 46, 47]),),
 array([9.99999881e-01, 2.27159488e-01, 1.00000000e+00, 2.27159268e-01,
        7.70674384e-01, 2.21044303e-01, 7.70674505e-01, 2.21044355e-01,
        5.99413238e-01, 3.88503275e-01, 5.99413243e-01, 3.88503408e-01,
        4.14197737e-01, 3.25603159e-01, 4.14197652e-01, 3.25603083e-01,
        5.32381621e-01, 2.69214730e-01, 5.32382115e-01, 2.69215431e-01,
        7.31269881e-01, 2.30185510e-01, 7.31269580e-01, 2.30186889e-01,
        7.52686122e-01, 2.07877661e-01, 7.52686226e-01, 2.07878370e-01,
        9.33114070e-01, 2.23977107e-01, 9.33114685e-01, 2.23977466e-01,
        4.06422128e-01, 2.79129787e-01, 4.06421867e-01, 2.79130203e-01,
        4.72363823e-01, 1.13901664e-01, 4.72363851e-01, 1.13901932e-01,
        6.65522954e-01, 1.10818149e-01, 6.65522857e-01, 1.10817969e-01,
        3.77989990e-01, 0.00000000e+00, 3.77990120e-01, 5.68245090e-